In [1]:
import torch
import numpy as np
from arsf_envi_reader import envi_header
import shutil
import os
import json
import math
import affine
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr
from scipy.optimize import curve_fit
from tqdm import tqdm
# import multiprocess as mp
# from numpy import trapz

In [2]:
aviris_img = gdal.Open(r'E:\wenqu\aviris\clipped_aviris\site1b\site1b_aviris_clip').ReadAsArray() 

In [3]:
print(aviris_img.shape)

(425, 122, 93)


In [4]:
inDs = gdal.Open(r'E:\wenqu\aviris\clipped_aviris\site1b\site1b_aviris_clip')

In [5]:
pj = inDs.GetProjection()  ## projection
gt = inDs.GetGeoTransform()  ## geotransform
col = inDs.GetRasterBand(1).XSize
row = inDs.GetRasterBand(1).YSize

In [7]:
d15n_trait = (-0.339880*aviris_img[37, :, :] + -1.433534*aviris_img[9, :, :] + -1.718473*aviris_img[25, :, :]
              + -0.703500*aviris_img[110, :, :] + 0.042741*aviris_img[121, :, :] + -1.193126*aviris_img[13, :, :] 
              + -0.478965*aviris_img[12, :, :] + -2.593645*aviris_img[90, :, :] + 0.133993*aviris_img[24, :, :] 
              + 0.317655*aviris_img[13, :, :] + 0.664927*aviris_img[22, :, :] + 0.094452*aviris_img[106, :, :]
              + 0.672444*aviris_img[108, :, :] + 0.109911*aviris_img[17, :, :] + 1.312284*aviris_img[10, :, :]
              + 2.030740*aviris_img[97, :, :] + -1.045017*aviris_img[8, :, :] + 0.948916*aviris_img[6, :, :]
              + 2.036864*aviris_img[100, :, :] + -0.709171*aviris_img[14, :, :] + 0.042938*aviris_img[2, :, :]
              + 1.465405*aviris_img[7, :, :] + -2.754928*aviris_img[98, :, :] + -0.156157*aviris_img[112, :, :] 
              + 1.138128*aviris_img[119, :, :] + 3.003813*aviris_img[107, :, :] + -0.672334*aviris_img[117, :, :] + -1.292843)

In [9]:
driver = gdal.GetDriverByName('GTiff')  ## driver for output file type; "GTiff" for geotiff, "RST" for rst
bands =1
outName = "aviris_d15n_spectra.tif"
outPath ='E:/wenqu/aviris/trait_map'
out_ds = driver.Create(os.path.join(outPath, outName),col,row,bands,gdal.GDT_Float32)
out_ds.GetRasterBand(1).WriteArray(d15n_trait)
out_ds.SetProjection(pj)
out_ds.SetGeoTransform(gt)
del out_ds

# d15n std

In [60]:
aviris_img = gdal.Open(r'E:\wenqu\aviris\clipped_aviris\site1b\site1b_aviris_clip').ReadAsArray() 
inDs = gdal.Open(r'E:\wenqu\aviris\clipped_aviris\site1b\site1b_aviris_clip')
pj = inDs.GetProjection()  ## projection
gt = inDs.GetGeoTransform()  ## geotransform
col = inDs.GetRasterBand(1).XSize
row = inDs.GetRasterBand(1).YSize

In [61]:
band_list =  ['band_283', 'band_148', 'band_354', 'band_91', 'band_17',
       'band_393', 'band_382', 'band_404', 'band_183', 'band_368',
       'band_380', 'band_184', 'band_126', 'band_72', 'band_106',
       'band_111', 'band_41', 'band_237', 'band_129', 'band_77',
       'band_79', 'band_379', 'band_33', 'band_158', 'band_99',
       'band_359', 'band_32', 'band_61', 'band_396', 'band_335',
       'band_92', 'band_139', 'band_50', 'band_144', 'band_37',
       'band_261', 'band_176', 'band_153', 'band_267', 'band_55',
       'band_64', 'band_276', 'band_80', 'band_63', 'band_133',
       'band_101', 'band_70', 'band_97', 'band_367', 'band_51', 'band_27',
       'band_394', 'band_217', 'band_227', 'band_167', 'band_78',
       'band_49', 'band_370', 'band_363', 'band_90', 'band_147',
       'band_60', 'band_333', 'band_56', 'band_112', 'band_103',
       'band_329', 'band_30', 'band_35', 'band_128', 'band_391',
       'band_331', 'band_69', 'band_16', 'band_388', 'band_114',
       'band_127', 'band_48', 'band_171', 'band_387', 'band_178',
       'band_68', 'band_141', 'band_118', 'band_268', 'band_140',
       'band_249', 'band_96', 'band_168', 'band_277', 'band_154',
       'band_223', 'band_46', 'band_259']

In [62]:
band_number = []
for i in range(94):
    number = int(band_list[i].split('_')[1])
    band_number.append(number)
#     print(band_number.datatype)

In [63]:
print(band_number)

[283, 148, 354, 91, 17, 393, 382, 404, 183, 368, 380, 184, 126, 72, 106, 111, 41, 237, 129, 77, 79, 379, 33, 158, 99, 359, 32, 61, 396, 335, 92, 139, 50, 144, 37, 261, 176, 153, 267, 55, 64, 276, 80, 63, 133, 101, 70, 97, 367, 51, 27, 394, 217, 227, 167, 78, 49, 370, 363, 90, 147, 60, 333, 56, 112, 103, 329, 30, 35, 128, 391, 331, 69, 16, 388, 114, 127, 48, 171, 387, 178, 68, 141, 118, 268, 140, 249, 96, 168, 277, 154, 223, 46, 259]


In [64]:
coef = [ 3.58183232,  3.30336088,  2.66743832, -2.52382841,  2.41633271,
       -2.40942019, -2.39000564, -2.35527338, -2.31753982,  2.30373874,
       -2.28362758, -2.26561998,  2.23648404, -2.22025928, -2.19360387,
       -2.1874008 ,  2.18511885,  2.17455071,  2.04615177,  1.97488939,
        1.96586522, -1.93179036, -1.8788226 ,  1.79634307, -1.79559493,
        1.77470757, -1.71873037,  1.71790574, -1.69498321,  1.69476184,
       -1.68269406,  1.66822559, -1.65649309, -1.65488277,  1.6443052 ,
        1.64305868,  1.6420349 ,  1.61480046, -1.58375664, -1.57540667,
        1.56794913, -1.56381293,  1.55937425,  1.5575692 ,  1.54973848,
       -1.54474313, -1.53615608, -1.50715797,  1.50612984, -1.49023006,
        1.48059113, -1.47658102, -1.44327508, -1.41688514, -1.40419127,
        1.40254403, -1.37536906,  1.37005895,  1.35572234,  1.34302916,
        1.33745711, -1.32659841, -1.32565808, -1.32183305, -1.31704911,
       -1.30921445, -1.30110887, -1.29667273,  1.29169568, -1.27801344,
        1.26639702, -1.26306698,  1.23484732,  1.2314902 ,  1.22429347,
        1.21514663,  1.18215109, -1.16226568, -1.14501289, -1.13527966,
       -1.13506537,  1.132024  ,  1.13045012,  1.12284045, -1.11518405,
        1.10516282,  1.10354787,  1.098394  , -1.08109405,  1.06473185,
       -1.05887977,  1.05756798,  1.05680724,  1.05509116]

In [65]:
print(len(coef))

94


In [66]:
coefficients = dict(zip(band_number, coef))

In [67]:
# Initialize an array of zeros with the same shape as one band to store the result
aviris_d15n_std = np.zeros_like(aviris_img[0, :, :])

# Loop through the dictionary to compute the result
for band_index, coef in coefficients.items():
    aviris_d15n_std += coef * aviris_img[band_index, :, :]

print(aviris_d15n_std)

[[-30042.77 -30042.77 -30042.77 ... -30042.77 -30042.77 -30042.77]
 [-30042.77 -30042.77 -30042.77 ... -30042.77 -30042.77 -30042.77]
 [-30042.77 -30042.77 -30042.77 ... -30042.77 -30042.77 -30042.77]
 ...
 [-30042.77 -30042.77 -30042.77 ... -30042.77 -30042.77 -30042.77]
 [-30042.77 -30042.77 -30042.77 ... -30042.77 -30042.77 -30042.77]
 [-30042.77 -30042.77 -30042.77 ... -30042.77 -30042.77 -30042.77]]


In [68]:
aviris_d15n_std.shape

(122, 93)

In [69]:
driver = gdal.GetDriverByName('GTiff')  ## driver for output file type; "GTiff" for geotiff, "RST" for rst
bands =1
outName = "site1b_aviris_d15n_std.tif"
outPath ='E:/wenqu/aviris/trait_map'
out_ds = driver.Create(os.path.join(outPath, outName),col,row,bands,gdal.GDT_Float32)
out_ds.GetRasterBand(1).WriteArray(aviris_d15n_std)
out_ds.SetProjection(pj)
out_ds.SetGeoTransform(gt)
del out_ds